In [1]:
import os
import glob
import pandas as pd
import pymongo
import json
from sqlalchemy import create_engine
import numpy as np

## Extract Stocks

In [7]:
#Get list of stock names
os.listdir("../etl-project/RawStock")
stock_filename = os.listdir("../etl-project/RawStock")
stock_list=[]
for i in stock_filename:
    if i[-4:] =='.txt':
        stock_list.append(i)

stock_filename=stock_list

In [8]:
stock_data = [pd.read_csv(f'../etl-project/RawStock/{file}') for file in stock_filename]

In [9]:
print(stock_filename)

['ibm.us.txt', 'ko.us.txt', 'intc.us.txt', 'wmt.us.txt', 'mcd.us.txt', 'nke.us.txt', 'msft.us.txt', 'xom.us.txt', 'ba.us.txt', 'jpm.us.txt', 'wba.us.txt', 'v.us.txt', 'utx.us.txt', 'dis.us.txt', 'aapl.us.txt', 'mmm.us.txt', 'mrk.us.txt', 'gs.us.txt', 'unh.us.txt', 'hd.us.txt', 'jnj.us.txt', 'csco.us.txt', 'cvx.us.txt', 'vz.us.txt', 'cat.us.txt', 'trv.us.txt', 'axp.us.txt', 'pg.us.txt', 'pfe.us.txt']


## Transform Stocks 

In [10]:
clean_ticker = [ticker[:-7] for ticker in stock_filename]
clean_ticker.sort()
print(clean_ticker)

['aapl', 'axp', 'ba', 'cat', 'csco', 'cvx', 'dis', 'gs', 'hd', 'ibm', 'intc', 'jnj', 'jpm', 'ko', 'mcd', 'mmm', 'mrk', 'msft', 'nke', 'pfe', 'pg', 'trv', 'unh', 'utx', 'v', 'vz', 'wba', 'wmt', 'xom']


In [11]:
stock_company = {
    "Ticker":clean_ticker,
    "Company Name": ["Apple","American Express","Boeing","Caterpillar","Cisco","Chevron","Disney","Goldman Sachs","Home Depot","IBM","Intel","Johnson & Johnson","JP Morgan Chase","Coca-Cola","McDonalds","3M","Merck","Microsoft","Nike","Pfizer","Proctor & Gamble","Travelers Companies","United Health","United Technologies","Visa","Verizon","Walgreens","Walmart","Exxon Mobil"]
}

stock_company = pd.DataFrame(stock_company)

In [12]:
for x in np.arange(len(stock_data)):
    stock_data[x]["Ticker"] = clean_ticker[x]

In [13]:
stocks = pd.concat(stock_data, axis=0)

In [14]:
stocks = stocks[["Ticker","Date","Open","Close"]]

In [15]:
stocks["Daily Percent"] = ((stocks["Close"]-stocks["Open"])/stocks["Open"])*100

In [16]:
stocks = pd.merge(stocks,stock_company, on="Ticker")

In [17]:
stocks.sample(20)

,Ticker,Date,Open,Close,Daily Percent,Company Name
196168,pfe,1998-08-03,23.6270,23.2750,-1.489821,Pfizer
274502,wmt,1997-08-08,29.3160,28.8740,-1.507709,Walmart
177108,mrk,2005-04-11,26.3000,26.0580,-0.920152,Merck
41046,cat,1985-10-14,1.3090,1.3262,1.313980,Caterpillar
214634,trv,1996-04-29,4.7839,4.7753,-0.179770,Travelers Companies
194290,pfe,1991-02-27,3.0674,3.1273,1.952794,Pfizer
209448,pg,2003-05-23,43.3540,43.2090,-0.334456,Proctor & Gamble
219226,trv,2014-07-29,23.2520,23.0990,-0.658008,Travelers Companies
178272,mrk,2009-11-20,27.8750,28.6700,2.852018,Merck
5743,aapl,1984-11-15,25.5760,25.5260,-0.195496,Apple


## Load Stocks

In [18]:
#Load into MongoDB
stock_data='stocks.csv'

stocks.to_csv(stock_data)

In [19]:
client = pymongo.MongoClient('localhost', 27017)
mng_db = client['FinanceDB'] 
collection_name = 'Stock_Data' 
db = mng_db[collection_name]

data = pd.read_csv(stock_data)
data_json = json.loads(data.to_json(orient='records'))
db.insert_many(data_json)

In [26]:
list(mng_db.Stock_Data.find())

[{'_id': ObjectId('5e950e9e7605338a4e84c2b1'),
  'Unnamed: 0': 0,
  'Ticker': 'aapl',
  'Date': '1962-01-02',
  'Open': 6.413,
  'Close': 6.3378,
  'Daily Percent': -1.1726181194,
  'Company Name': 'Apple'},
 {'_id': ObjectId('5e950e9e7605338a4e84c2b2'),
  'Unnamed: 0': 1,
  'Ticker': 'aapl',
  'Date': '1962-01-03',
  'Open': 6.3378,
  'Close': 6.3963,
  'Daily Percent': 0.9230332292,
  'Company Name': 'Apple'},
 {'_id': ObjectId('5e950e9e7605338a4e84c2b3'),
  'Unnamed: 0': 2,
  'Ticker': 'aapl',
  'Date': '1962-01-04',
  'Open': 6.3963,
  'Close': 6.3295,
  'Daily Percent': -1.044353767,
  'Company Name': 'Apple'},
 {'_id': ObjectId('5e950e9e7605338a4e84c2b4'),
  'Unnamed: 0': 3,
  'Ticker': 'aapl',
  'Date': '1962-01-05',
  'Open': 6.3211,
  'Close': 6.2041,
  'Daily Percent': -1.8509436649,
  'Company Name': 'Apple'},
 {'_id': ObjectId('5e950e9e7605338a4e84c2b5'),
  'Unnamed: 0': 4,
  'Ticker': 'aapl',
  'Date': '1962-01-08',
  'Open': 6.2041,
  'Close': 6.087,
  'Daily Percent': -1

In [20]:
#Load into SQL
rds_connection_string = "postgres:postgres@localhost:5433/FinanceDB"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [21]:
stock=pd.read_csv('stocks.csv', index_col=0)
stock.head()

,Ticker,Date,Open,Close,Daily Percent,Company Name
0,aapl,1962-01-02,6.4130,6.3378,-1.172618,Apple
1,aapl,1962-01-03,6.3378,6.3963,0.923033,Apple
2,aapl,1962-01-04,6.3963,6.3295,-1.044354,Apple
3,aapl,1962-01-05,6.3211,6.2041,-1.850944,Apple
4,aapl,1962-01-08,6.2041,6.0870,-1.887462,Apple


In [22]:
stock.to_sql(name='stock_data', con=engine, if_exists='append', index=False) 

In [23]:
pd.read_sql_query('select * from stock_data', con=engine).head()

,Ticker,Date,Open,Close,Daily Percent,Company Name
0,aapl,1962-01-02,6.4130,6.3378,-1.172618,Apple
1,aapl,1962-01-03,6.3378,6.3963,0.923033,Apple
2,aapl,1962-01-04,6.3963,6.3295,-1.044354,Apple
3,aapl,1962-01-05,6.3211,6.2041,-1.850944,Apple
4,aapl,1962-01-08,6.2041,6.0870,-1.887462,Apple
